In [92]:
import pandas as pd
import numpy as np
import uuid
import json
from pathlib import Path

In [93]:
SHORT_WINDOW = 10
LONG_WINDOW = 25
RISK_PCT = 0.05

class DefaultStrategy:
    def __init__(self):
        self.initialized = False
        self.short_window = SHORT_WINDOW
        self.long_window = LONG_WINDOW
        self.risk_pct = RISK_PCT

        self.price_history = {
            "token_1/fiat": [],
            "token_2/fiat": [],
            "token_1/token_2": []
        }

        # Seguimiento de últimas compras para evitar sobreventas
        self.last_buy_qty_token1 = 0
        self.last_buy_qty_token2 = 0

    def on_data(self, market_data, balances):
        orders = []

        for pair, data in market_data.items():
            if pair in self.price_history:
                self.price_history[pair].append(data["close"])
                if len(self.price_history[pair]) > self.long_window:
                    self.price_history[pair] = self.price_history[pair][-self.long_window:]

        # Asegurarse de tener suficientes datos
        for prices in self.price_history.values():
            if len(prices) < self.long_window:
                return orders

        fee = market_data["fee"]

        # === token_1/fiat ===
        if "token_1/fiat" in self.price_history:
            prices = self.price_history["token_1/fiat"]
            short_ma = np.mean(prices[-self.short_window:])
            long_ma = np.mean(prices[-self.long_window:])
            price = prices[-1]

            if short_ma > long_ma:
                fiat_balance = balances["fiat"]
                qty = (self.risk_pct * fiat_balance) / (price * (1 + fee))
                if qty * price * (1 + fee) <= fiat_balance:
                    orders.append({"pair": "token_1/fiat", "side": "buy", "qty": qty})
                    self.last_buy_qty_token1 = qty

            elif short_ma < long_ma:
                token1_balance = balances["token_1"]
                qty = min(self.last_buy_qty_token1, token1_balance)
                if qty > 0:
                    orders.append({"pair": "token_1/fiat", "side": "sell", "qty": qty})
                    self.last_buy_qty_token1 = 0  # Reset tras vender

        # === token_2/fiat ===
        if "token_2/fiat" in self.price_history:
            prices = self.price_history["token_2/fiat"]
            short_ma = np.mean(prices[-self.short_window:])
            long_ma = np.mean(prices[-self.long_window:])
            price = prices[-1]

            if short_ma > long_ma:
                fiat_balance = balances["fiat"]
                qty = (self.risk_pct * fiat_balance) / (price * (1 + fee))
                if qty * price * (1 + fee) <= fiat_balance:
                    orders.append({"pair": "token_2/fiat", "side": "buy", "qty": qty})
                    self.last_buy_qty_token2 = qty

            elif short_ma < long_ma:
                token2_balance = balances["token_2"]
                qty = min(self.last_buy_qty_token2, token2_balance)
                if qty > 0:
                    orders.append({"pair": "token_2/fiat", "side": "sell", "qty": qty})
                    self.last_buy_qty_token2 = 0

        return orders


In [94]:
def run_backtest(combined_data: pd.DataFrame, fee: float, balances: dict[str, float]) -> pd.DataFrame:
    """Run a backtest with multiple trading pairs, applying strategy decisions and simulating balances."""
    # Record initial balances
    initial_balances = balances.copy()

    # Ordenar por timestamp
    combined_data.sort_values("timestamp", inplace=True)
    result = pd.DataFrame(columns=["id", "timestamp", "pair", "side", "qty"])

    for timestamp, group in combined_data.groupby('timestamp'):
        market_data = {"fee": fee}
        for _, row in group.iterrows():
            pair = row['symbol']
            market_data[pair] = row.to_dict()

        # Obtener órdenes de la estrategia
        orders = strategy.on_data(market_data, balances)

        # Agregar arbitraje (dentro del bucle, usando balances actualizados)
        if all(p in market_data for p in ["token_1/fiat", "token_2/fiat", "token_1/token_2"]):
            token1_price = market_data["token_1/fiat"]["close"]
            token2_price = market_data["token_2/fiat"]["close"]
            token1_token2_price = market_data["token_1/token_2"]["close"]

            implied_price = token1_price / token2_price
            total_fee_multiplier = (1 + fee) ** 2
            required_spread = total_fee_multiplier

            # Arbitraje: comprar token_1 con token_2
            if token1_token2_price < implied_price / required_spread:
                qty = 0.01
                required_token2 = qty * token1_token2_price * (1 + fee)
                if balances["token_2"] >= required_token2:
                    print(f"[ARBITRAJE COMPRA] {timestamp}")
                    orders.append({"pair": "token_1/token_2", "side": "buy", "qty": qty})

            # Arbitraje: vender token_1 por token_2
            elif token1_token2_price > implied_price * required_spread:
                qty = min(0.01, balances["token_1"])
                if qty > 0:
                    print(f"[ARBITRAJE VENTA] {timestamp}")
                    orders.append({"pair": "token_1/token_2", "side": "sell", "qty": qty})

        # Simular ejecución de órdenes
        for order in orders:
            pair = order["pair"]
            side = order["side"]
            qty = order["qty"]
            price = market_data[pair]["close"]
            base, quote = pair.split("/")

            if side == "buy":
                cost = qty * price * (1 + fee)
                if balances.get(quote, 0) >= cost:
                    balances[quote] -= cost
                    balances[base] = balances.get(base, 0) + qty
                else:
                    continue  # saldo insuficiente

            elif side == "sell":
                if balances.get(base, 0) >= qty:
                    balances[base] -= qty
                    revenue = qty * price * (1 - fee)
                    balances[quote] = balances.get(quote, 0) + revenue
                else:
                    continue  # saldo insuficiente

            order["timestamp"] = timestamp
            order["id"] = str(uuid.uuid4())
            result = pd.concat([result, pd.DataFrame([order])], ignore_index=True)

    return result


In [95]:
with open("./kaggle/input/crypto-trading-hackathon-2025/hyperparameters.json") as f:
    HYPERPARAMETERS = json.load(f)
    
FEE = HYPERPARAMETERS.get("fee", 3.0)
BALANCE_FIAT = HYPERPARAMETERS.get("fiat_balance", 1000.0)
BALANCE_TOKEN1 = HYPERPARAMETERS.get("token1_balance", 0.0)
BALANCE_TOKEN2 = HYPERPARAMETERS.get("token2_balance", 0.0)
OUTPUT = "submission.csv"

combined_data = pd.read_csv("./kaggle/input/crypto-trading-hackathon-2025/test.csv")

# Run the backtest on the provided test data with a fee of 0.02% and initial balances of 10,000 fiat, and 0 token_1 and token_2
result = run_backtest(combined_data, FEE, {
    "fiat": BALANCE_FIAT,
    "token_1": BALANCE_TOKEN1,
    "token_2": BALANCE_TOKEN2,
})

# Output the backtest result to a CSV file for submission
result.to_csv(OUTPUT, index=False)

/tmp/ipykernel_111491/3501155753.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, pd.DataFrame([order])], ignore_index=True)


In [96]:
import pandas as pd

df = pd.read_csv("submission.csv")
summary = df.groupby(['pair', 'side'])['qty'].sum()
print(summary)


pair          side
token_1/fiat  buy     0.552505
              sell    0.512104
token_2/fiat  buy     0.009638
              sell    0.009253
Name: qty, dtype: float64
